In [1]:
import torch
from transformer_lens import HookedTransformer
from transformer_lens import utils

from functools import partial

torch.set_grad_enabled(False)



In [63]:
model: HookedTransformer = HookedTransformer.from_pretrained('gpt2-small', device='cpu')


Loaded pretrained model gpt2-small into HookedTransformer


In [65]:
layer = 1
# prompt_pos = "Yes, I talk about wedding constantly"
# prompt_neg = "I do not talk about wedding constantly"
prompt_pos = "Love "
prompt_neg = "Hate"

In [66]:
logits, cache = model.run_with_cache(prompt_pos)
h_p = cache["resid_post", layer]

logits, cache = model.run_with_cache(prompt_neg)
h_n = cache["resid_post", layer]

print(h_p.shape, h_n.shape)
steering = h_p - h_n
steering.shape

torch.Size([1, 3, 768]) torch.Size([1, 3, 768])


torch.Size([1, 3, 768])

In [73]:
c = 10

def residual_stream_patching_hook(
    resid,
    hook,
    layer: int
):
    # resid shape is (batch, pos, d_model)
    len_steering = steering.shape[1]
    # print(resid.shape, steering.shape)

    
    resid[:, :len_steering, :] =  resid[:, :len_steering, :] + c * steering

    return resid

In [74]:
text = "I hate you because"
n_samples = 5
hook_name = utils.get_act_name("resid_post", layer)

with model.hooks(fwd_hooks=[(hook_name, partial(residual_stream_patching_hook, layer=layer))]):
    # model('testing testing asdf;lkjasdf;l k')
    for i in range(n_samples):
        output = model.generate(text, prepend_bos=True, use_past_kv_cache=False, max_new_tokens=30)
        print(output)

  0%|          | 0/30 [00:00<?, ?it/s]

In [53]:
steering

tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 2.8142e-03,  2.1155e-01, -4.3170e+00,  ...,  2.4842e+00,
          -3.3969e+00, -1.9576e+00],
         [-3.8675e+00,  1.2957e+00,  3.1500e+00,  ...,  4.6703e-01,
          -7.2199e-01, -5.2960e+00],
         ...,
         [ 7.1123e-02,  6.5655e-01, -2.0395e-01,  ..., -1.1744e+00,
          -1.3301e+00,  1.2973e-01],
         [-2.7208e-01,  7.5982e-01,  9.4286e-02,  ..., -1.1328e-01,
          -3.2575e-01,  5.4140e-01],
         [-1.0943e+00,  1.7197e+00,  1.2230e+00,  ...,  1.7851e-01,
          -8.4527e-01,  6.5455e-01]]])